# Deep Learning Model for Fake News Detection

In [1]:
# Load and Prepare Data
import pandas as pd
from sklearn import preprocessing

data = pd.read_csv("news.csv")
data = data.drop(["Unnamed: 0"], axis=1)

le = preprocessing.LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])

In [2]:
# Tokenize and Pad Sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(data['title'])
sequences1 = tokenizer1.texts_to_sequences(data['title'])
padded1 = pad_sequences(sequences1, padding='post', truncating='post', maxlen=54)

training_size = 3000
test_portion = 0.1
split = int(test_portion * training_size)

training_sequences1 = padded1[split:training_size]
test_sequences1 = padded1[0:split]

labels = data['label'].tolist()
training_labels = labels[split:training_size]
test_labels = labels[0:split]

2024-05-30 17:55:09.262228: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Load GloVe Embeddings
import numpy as np

embeddings_index = {}
with open('glove.6B.50d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

vocab_size1 = len(tokenizer1.word_index)
embedding_dim = 50
embeddings_matrix = np.zeros((vocab_size1 + 1, embedding_dim))

for word, i in tokenizer1.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector

In [4]:
# Build and Train the Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, MaxPooling1D, LSTM, Dense

model = Sequential([
    Embedding(vocab_size1 + 1, embedding_dim, input_length=54, weights=[embeddings_matrix], trainable=False),
    Dropout(0.2),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(pool_size=4),
    LSTM(64),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

num_epochs = 50
history = model.fit(np.array(training_sequences1), np.array(training_labels), epochs=num_epochs,
                    validation_data=(np.array(test_sequences1), np.array(test_labels)), verbose=2)

2024-05-30 17:55:15.275628: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/50


2024-05-30 17:55:15.556958: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-30 17:55:15.558604: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-30 17:55:15.559572: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

85/85 - 8s - loss: 0.6419 - accuracy: 0.6311 - val_loss: 0.5770 - val_accuracy: 0.6933 - 8s/epoch - 91ms/step
Epoch 2/50
85/85 - 4s - loss: 0.5689 - accuracy: 0.7063 - val_loss: 0.5290 - val_accuracy: 0.7333 - 4s/epoch - 42ms/step
Epoch 3/50
85/85 - 4s - loss: 0.5307 - accuracy: 0.7311 - val_loss: 0.5270 - val_accuracy: 0.7133 - 4s/epoch - 46ms/step
Epoch 4/50
85/85 - 4s - loss: 0.4931 - accuracy: 0.7600 - val_loss: 0.5186 - val_accuracy: 0.7433 - 4s/epoch - 45ms/step
Epoch 5/50
85/85 - 4s - loss: 0.4440 - accuracy: 0.7956 - val_loss: 0.5101 - val_accuracy: 0.7333 - 4s/epoch - 45ms/step
Epoch 6/50
85/85 - 4s - loss: 0.4185 - accuracy: 0.8070 - val_loss: 0.4952 - val_accuracy: 0.7300 - 4s/epoch - 48ms/step
Epoch 7/50
85/85 - 4s - loss: 0.3667 - accuracy: 0.8348 - val_loss: 0.5979 - val_accuracy: 0.7200 - 4s/epoch - 48ms/step
Epoch 8/50
85/85 - 4s - loss: 0.3607 - accuracy: 0.8404 - val_loss: 0.5790 - val_accuracy: 0.7133 - 4s/epoch - 52ms/step
Epoch 9/50
85/85 - 5s - loss: 0.3040 - accu

In [5]:
# Detect Fake News
sample_text = "Karry to go to France in gesture of sympathy"
sample_sequence = tokenizer1.texts_to_sequences([sample_text])[0]
sample_padded = pad_sequences([sample_sequence], maxlen=54, padding='post', truncating='post')

prediction = model.predict(sample_padded, verbose=0)[0][0]
if prediction >= 0.5:
    print("This news is True")
else:
    print("This news is false")

This news is false


2024-05-30 17:58:39.924539: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-30 17:58:39.927236: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-30 17:58:39.928573: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus